# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

### Imports

In [320]:
import pandas as pd
import numpy as np
from mpmath import *

In [321]:
leitura = pd.ExcelFile('tweets_Nike_1.xlsx')

In [322]:
tweets_treinamento = pd.read_excel(leitura, 'Treinamento')

In [323]:
tweets_teste = pd.read_excel(leitura, 'Teste') 

In [324]:
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(':', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('@', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('*', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('“', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('.', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(')', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('(', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('_', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(';', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace(',', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('&', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('/', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('#', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('%', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('$', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("!", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("?", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("-", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("+", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("=", '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace('rt', '', regex=False)
tweets_treinamento.Treinamento = tweets_treinamento.Treinamento.str.replace("\n", '', regex=False)

In [325]:
tweets_teste.Teste = tweets_teste.Teste.str.replace(':', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('@', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('*', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('“', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('.', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(')', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('(', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('_', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(';', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace(',', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('&', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('/', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('#', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('%', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('$', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("!", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("?", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("-", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("+", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("=", '')
tweets_teste.Teste = tweets_teste.Teste.str.replace('rt', '')
tweets_teste.Teste = tweets_teste.Teste.str.replace("\n", '')

In [326]:
rel_treinamento = tweets_treinamento[(tweets_treinamento['classificacao'] == 1)]
irrel_treinamento = tweets_treinamento[(tweets_treinamento['classificacao'] == 0)]
tweets_treinamento['Treinamento'] = tweets_treinamento

rel_teste = tweets_teste[(tweets_teste['classificacao'] == 1)]
irrel_teste = tweets_teste[(tweets_teste['classificacao'] == 0)]
tweets_teste['Teste'] = tweets_teste

In [327]:
lrel= []
lirrel= []
ltodos = []

for a in tweets_treinamento['Treinamento']:
    a = a.lower()
    
for a in rel_treinamento['Treinamento']:
    lrel.append(a.split())
    
for a in irrel_treinamento['Treinamento']:
    lirrel.append(a.split())
    
for a in tweets_treinamento['Treinamento']:
    ltodos.append(a.split())  

In [328]:
n_irrel = tweets_treinamento.classificacao[tweets_treinamento.classificacao == 0].count()
n_rel = tweets_treinamento.classificacao[tweets_treinamento.classificacao == 1].count()
total_n = n_rel + n_irrel

prob_irrelevante = n_irrel/ total
prob_relevante = n_rel/ total

dic_rel = {}
dic_irrel = {}
dic_tweet = {}

Ln_rel = []
Ln_irrel = []
TD = []

for f in lrel:
    for p in f:
        Ln_rel.append(p)
        dic_rel[p] = Ln_rel.count(p)

for f in lirrel:
    for p in f:
        Ln_irrel.append(p)
        dic_irrel[p] = Ln_irrel.count(p)

for f in ltodos:
    for p in f:
        TD.append(p)
        dic_tweet[p] = TD.count(p)

In [329]:
P_relevante = len(rel)/len(tweets_treinamento)
P_irrelevante = len(irrel)/len(tweets_treinamento)

In [330]:
print(len(TD))
print(len(Ln_rel))
print(len(Ln_irrel))

5380
378
5002


In [331]:
new= list(set(TD))
print(len(new))
print(len(TD))

1593
5380


In [332]:
p_r = {}
p_ir = {}

for pl in Ln_rel:
    p_r[pl] = (dic_rel[pl] + 1)/ (len(new) + len(Ln_rel))

for pl in Ln_irrel:
    p_ir[pl] = (dic_irrel[pl] + 1)/ (len(new) + len(Ln_irrel))


In [333]:
r = len(Ln_rel)/len(TD)
ir = len(Ln_irrel)/len(TD)

In [334]:
soma_rel = sum(p_r.values())
soma_irrel = sum(p_ir.values())

print(soma_rel + soma_irrel)

1.2827723753331317


In [335]:
classific = []
for frase in tweets_teste['Teste']:
    palavra_frase2 = []
    palavra_frase2.append(frase.split())
    p_fraseir = 1
    p_pal_ir = 0
    p_fraser = 1
    p_pal_r = 0
    for a in palavra_frase2:
        for word in a:
            palavra_frase = []
            palavra_frase.append(word)                          
            for palavra in palavra_frase:
                p_fraseir = 1
                p_pal_ir = 0
                p_fraser = 1
                p_pal_r = 0
                #print(palavra)
                if palavra in p_r:
                    p_pal_r = p_r[palavra]
                    p_fraser *= p_pal_r
                elif palavra in p_ir:
                    p_pal_ir = p_ir[palavra]
                    p_fraseir *= p_pal_ir
                else:
                    p_pal_ir = 1/(len(Ln_irrel) + len(new))
                    p_pal_r = 1/(len(Ln_rel) + len(new))
                    p_fraser *= p_pal_r
                    p_fraseir *= p_pal_ir
    if p_fraser > p_fraseir:
        classific.append(1)
    else:
        classific.append(0)
        
tweets_teste['Chute'] = classific

vp = 0
vn = 0
fp = 0
fn = 0

for i in range(len(tweets_teste)):
    if tweets_teste['Chute'][i] == 1 and tweets_teste['classificacao'][i] == 1:
        vp += 1
    elif tweets_teste['Chute'][i] == 0 and tweets_teste['classificacao'][i] == 1:
        fn += 1
    elif tweets_teste['Chute'][i] == 0 and tweets_teste['classificacao'][i] == 0:
        vn += 1
    else:
        fp += 1

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [336]:
print('A porcentagem de positivos verdadeiros é:', vp/2,'%')
print('A porcentagem de negativos verdadeiros é:', vn/2,'%')
print('A porcentagem de positivos falsos é:', fp/2,'%')
print('A porcentagem de negativos falsos é:', fn/2,'%')

A porcentagem de positivos verdadeiros é: 7.0 %
A porcentagem de negativos verdadeiros é: 11.0 %
A porcentagem de positivos falsos é: 77.5 %
A porcentagem de negativos falsos é: 4.5 %


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
